<a href="https://colab.research.google.com/github/Maria-lin/How-to-make-notebook-in-dataiku/blob/main/Detection_Anomalies_DAB_Professionnel_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏦 Détection d'Anomalies sur les Distributeurs Automatiques (DAB)
## Analyse Senior - Approche Multi-Algorithmes

---

### 📋 Objectifs
- Détecter comportements anormaux des DAB (fraude, dysfonctionnement, usage atypique)
- Approche non-supervisée avec validation croisée multi-algorithmes
- Interprétabilité et recommandations actionnables

### 🎯 Méthodologie
1. Audit qualité et EDA approfondie
2. Feature engineering métier
3. Détection multi-algorithmes (IF + LOF + One-Class SVM)
4. Consensus et scoring
5. Interprétabilité et validation
6. Export et recommandations

In [ ]:
# ============================================================================
# IMPORTS ET CONFIGURATION
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# ML & Stats
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from scipy import stats

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("="*80)
print(" "*25 + "✅ ENVIRONNEMENT CONFIGURÉ")
print("="*80)
print(f"📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎲 Random State : {RANDOM_STATE}")

---
# 📥 SECTION 1 : Chargement des Données

⚠️ **ACTION REQUISE** : Modifiez `DATASET_NAME` avec le nom de votre dataset Dataiku

In [ ]:
import dataiku

# 🔧 PARAMÈTRE À CONFIGURER
DATASET_NAME = "VOTRE_DATASET_DAB"  # ⚠️ MODIFIER ICI

try:
    dataset = dataiku.Dataset(DATASET_NAME)
    df = dataset.get_dataframe()

    print(f"✅ Dataset chargé : {DATASET_NAME}")
    print(f"📊 Dimensions : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
    print(f"💾 Mémoire : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")

    display(df.head(10))

except Exception as e:
    print(f"❌ Erreur : {e}")
    print("\n💡 Vérifiez :")
    print("   1. Le nom du dataset est correct")
    print("   2. Vous avez les permissions")
    raise

---
# 🔍 SECTION 2 : Audit Qualité Complet

In [ ]:
print("="*80)
print(" "*25 + "📊 AUDIT QUALITÉ")
print("="*80)

# 2.1 Structure
print("\n🔎 Informations générales :")
df.info()

# 2.2 Valeurs manquantes
print("\n🕳️ Valeurs manquantes :")
missing = pd.DataFrame({
    'nb_missing': df.isnull().sum(),
    'pct_missing': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('pct_missing', ascending=False)
missing = missing[missing['nb_missing'] > 0]

if len(missing) > 0:
    display(missing)
else:
    print("✅ Aucune valeur manquante")

# 2.3 Doublons
dup_rows = df.duplicated().sum()
print(f"\n🔄 Doublons de lignes : {dup_rows:,}")
if 'num_automate' in df.columns:
    dup_id = df.duplicated(subset=['num_automate']).sum()
    print(f"🔄 Doublons num_automate : {dup_id:,}")

# 2.4 Incohérences
print("\n⚙️ Contrôles de cohérence :")
checks = {}

if 'montant_total' in df.columns:
    checks['montant_negatif'] = (df['montant_total'] < 0).sum()
    checks['montant_zero'] = (df['montant_total'] == 0).sum()

if 'nb_total_de_retraits' in df.columns:
    checks['nb_negatif'] = (df['nb_total_de_retraits'] < 0).sum()
    checks['nb_zero'] = (df['nb_total_de_retraits'] == 0).sum()

if 'montant_total' in df.columns and 'nb_total_de_retraits' in df.columns:
    checks['montant_pos_nb_zero'] = ((df['montant_total'] > 0) & (df['nb_total_de_retraits'] == 0)).sum()
    checks['montant_zero_nb_pos'] = ((df['montant_total'] == 0) & (df['nb_total_de_retraits'] > 0)).sum()

checks_df = pd.DataFrame.from_dict(checks, orient='index', columns=['Nombre'])
checks_df['Pourcentage'] = (checks_df['Nombre'] / len(df) * 100).round(2)
display(checks_df.sort_values('Nombre', ascending=False))

---
# 📊 SECTION 3 : EDA - Analyse Exploratoire

In [ ]:
# 3.1 Variables catégorielles
cat_cols = [c for c in ['lib_site_implementation','type_carte','dab_hos_site','typ_gab'] if c in df.columns]

if cat_cols:
    print("🏷️ Variables catégorielles :\n")
    for col in cat_cols:
        print(f"\n{col} - {df[col].nunique()} modalités")
        display(df[col].value_counts().head(10))

In [ ]:
# 3.2 Variables numériques
num_cols = [c for c in ['montant_total','nb_total_de_retraits'] if c in df.columns]

if num_cols:
    print("🔢 Statistiques numériques :\n")
    stats_df = df[num_cols].describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]).T
    stats_df['skewness'] = df[num_cols].skew()
    stats_df['kurtosis'] = df[num_cols].kurtosis()
    display(stats_df)

    # Visualisations
    fig, axes = plt.subplots(len(num_cols), 3, figsize=(15, 5*len(num_cols)))
    if len(num_cols) == 1:
        axes = axes.reshape(1, -1)

    for idx, col in enumerate(num_cols):
        data = df[col].dropna()

        # Histogramme
        axes[idx,0].hist(data, bins=50, edgecolor='black', alpha=0.7)
        axes[idx,0].axvline(data.median(), color='red', linestyle='--', label=f'Médiane: {data.median():,.0f}')
        axes[idx,0].set_title(f'Distribution - {col}')
        axes[idx,0].legend()

        # Boxplot
        axes[idx,1].boxplot(data, vert=True)
        axes[idx,1].set_title(f'Boxplot - {col}')

        # Q-Q plot
        stats.probplot(data, dist="norm", plot=axes[idx,2])
        axes[idx,2].set_title(f'Q-Q Plot - {col}')

    plt.tight_layout()
    plt.show()

In [ ]:
# 3.3 Relation montant vs nombre de retraits
if 'montant_total' in df.columns and 'nb_total_de_retraits' in df.columns:
    plt.figure(figsize=(10, 6))

    if 'dab_hos_site' in df.columns:
        for cat in df['dab_hos_site'].dropna().unique():
            mask = df['dab_hos_site'] == cat
            plt.scatter(df.loc[mask, 'nb_total_de_retraits'],
                       df.loc[mask, 'montant_total'],
                       alpha=0.5, label=cat)
        plt.legend(title='Type DAB')
    else:
        plt.scatter(df['nb_total_de_retraits'], df['montant_total'], alpha=0.5)

    plt.xlabel('Nombre de retraits')
    plt.ylabel('Montant total (€)')
    plt.title('Relation Montant - Volume')
    plt.grid(alpha=0.3)
    plt.show()

    corr = df[['montant_total','nb_total_de_retraits']].corr().iloc[0,1]
    print(f"\n📊 Corrélation : {corr:.3f}")

---
# 🔨 SECTION 4 : Feature Engineering Métier

In [ ]:
print("="*80)
print(" "*20 + "🔨 CRÉATION DE FEATURES MÉTIER")
print("="*80)

df_enriched = df.copy()
features_created = []

# Feature 1: Montant moyen par retrait
if 'montant_total' in df.columns and 'nb_total_de_retraits' in df.columns:
    df_enriched['montant_moyen_par_retrait'] = np.where(
        df_enriched['nb_total_de_retraits'] > 0,
        df_enriched['montant_total'] / df_enriched['nb_total_de_retraits'],
        np.nan
    )
    features_created.append('montant_moyen_par_retrait')
    print("✅ montant_moyen_par_retrait = montant_total / nb_retraits")

# Feature 2: Ratio vs médiane (montant)
if 'montant_total' in df.columns:
    median_mt = df_enriched['montant_total'].median()
    df_enriched['ratio_montant_vs_median'] = df_enriched['montant_total'] / median_mt
    features_created.append('ratio_montant_vs_median')
    print(f"✅ ratio_montant_vs_median = montant / médiane ({median_mt:,.2f})")

# Feature 3: Ratio vs médiane (nombre)
if 'nb_total_de_retraits' in df.columns:
    median_nb = df_enriched['nb_total_de_retraits'].median()
    df_enriched['ratio_nb_vs_median'] = df_enriched['nb_total_de_retraits'] / median_nb
    features_created.append('ratio_nb_vs_median')
    print(f"✅ ratio_nb_vs_median = nb_retraits / médiane ({median_nb:,.0f})")

# Feature 4: Montant attendu vs observé
if 'montant_moyen_par_retrait' in df_enriched.columns:
    median_moy = df_enriched['montant_moyen_par_retrait'].median()
    df_enriched['montant_attendu'] = df_enriched['nb_total_de_retraits'] * median_moy
    df_enriched['ratio_observe_vs_attendu'] = np.where(
        df_enriched['montant_attendu'] > 0,
        df_enriched['montant_total'] / df_enriched['montant_attendu'],
        np.nan
    )
    features_created.extend(['montant_attendu', 'ratio_observe_vs_attendu'])
    print("✅ ratio_observe_vs_attendu = montant_observé / montant_attendu")

# Feature 5-6: Transformations log
if 'montant_total' in df.columns:
    df_enriched['log_montant'] = np.log1p(df_enriched['montant_total'])
    features_created.append('log_montant')
    print("✅ log_montant = log(1 + montant)")

if 'nb_total_de_retraits' in df.columns:
    df_enriched['log_nb_retraits'] = np.log1p(df_enriched['nb_total_de_retraits'])
    features_created.append('log_nb_retraits')
    print("✅ log_nb_retraits = log(1 + nb_retraits)")

# Feature 7: Z-score
if 'montant_total' in df.columns:
    mean_mt = df_enriched['montant_total'].mean()
    std_mt = df_enriched['montant_total'].std()
    if std_mt > 0:
        df_enriched['zscore_montant'] = (df_enriched['montant_total'] - mean_mt) / std_mt
        features_created.append('zscore_montant')
        print("✅ zscore_montant = (montant - moyenne) / std")

# Feature 8-10: Indicateurs binaires
if 'nb_total_de_retraits' in df.columns:
    p10 = df_enriched['nb_total_de_retraits'].quantile(0.1)
    p90 = df_enriched['nb_total_de_retraits'].quantile(0.9)
    df_enriched['is_low_activity'] = (df_enriched['nb_total_de_retraits'] <= p10).astype(int)
    df_enriched['is_high_activity'] = (df_enriched['nb_total_de_retraits'] >= p90).astype(int)
    features_created.extend(['is_low_activity', 'is_high_activity'])
    print(f"✅ is_low_activity (P10={p10:.0f}), is_high_activity (P90={p90:.0f})")

if 'dab_hos_site' in df.columns:
    df_enriched['is_hors_site'] = (df_enriched['dab_hos_site'] == 'H').astype(int)
    features_created.append('is_hors_site')
    print("✅ is_hors_site = 1 si hors site, 0 sinon")

print(f"\n📊 Total : {len(features_created)} features créées")

if features_created:
    print("\n📈 Statistiques des nouvelles features :")
    display(df_enriched[features_created].describe(percentiles=[.05,.25,.5,.75,.95]).T)

---
# ⚖️ SECTION 5 : Préparation et Normalisation

In [ ]:
# Sélection des features pour la modélisation
model_features = [
    'montant_total',
    'nb_total_de_retraits',
    'montant_moyen_par_retrait',
    'ratio_montant_vs_median',
    'ratio_nb_vs_median',
    'ratio_observe_vs_attendu',
    'log_montant',
    'log_nb_retraits',
    'zscore_montant',
    'is_low_activity',
    'is_high_activity',
    'is_hors_site'
]
model_features = [f for f in model_features if f in df_enriched.columns]

print(f"📋 Features sélectionnées ({len(model_features)}) :")
for i, f in enumerate(model_features, 1):
    print(f"  {i:2d}. {f}")

# Dataframe de modélisation
df_model = df_enriched[model_features].copy()

# Nettoyage
df_model.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in df_model.select_dtypes(include=[np.number]).columns:
    if df_model[col].isnull().any():
        df_model[col].fillna(df_model[col].median(), inplace=True)

print(f"\n✅ Dataset prêt : {df_model.shape[0]:,} × {df_model.shape[1]}")
print(f"💾 Mémoire : {df_model.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Normalisation avec RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(df_model)
df_scaled = pd.DataFrame(X_scaled, columns=df_model.columns, index=df_model.index)

print(f"\n⚖️ Normalisation effectuée (RobustScaler - robuste aux outliers)")

---
# 🤖 SECTION 6 : Détection d'Anomalies Multi-Algorithmes

## 6.1 Isolation Forest

In [ ]:
print("="*80)
print(" "*25 + "🌲 ISOLATION FOREST")
print("="*80)

# Paramètres
CONTAMINATION = 0.05  # 5% d'anomalies attendues

# Modèle
iso_forest = IsolationForest(
    n_estimators=500,
    contamination=CONTAMINATION,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Entraînement et prédiction
predictions_if = iso_forest.fit_predict(df_scaled)
scores_if = -iso_forest.score_samples(df_scaled)

# Résultats
df_enriched['if_anomaly'] = (predictions_if == -1).astype(int)
df_enriched['if_score'] = scores_if

n_anomalies_if = df_enriched['if_anomaly'].sum()
pct_if = (n_anomalies_if / len(df_enriched) * 100).round(2)

print(f"\n✅ Détection terminée")
print(f"🔴 Anomalies : {n_anomalies_if:,} ({pct_if}%)")
print(f"🟢 Normaux   : {len(df_enriched) - n_anomalies_if:,} ({100-pct_if}%)")

## 6.2 Local Outlier Factor (LOF)

In [ ]:
print("="*80)
print(" "*25 + "📍 LOCAL OUTLIER FACTOR")
print("="*80)

# Modèle LOF
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=CONTAMINATION,
    n_jobs=-1
)

# Prédiction
predictions_lof = lof.fit_predict(df_scaled)
scores_lof = -lof.negative_outlier_factor_

# Résultats
df_enriched['lof_anomaly'] = (predictions_lof == -1).astype(int)
df_enriched['lof_score'] = scores_lof

n_anomalies_lof = df_enriched['lof_anomaly'].sum()
pct_lof = (n_anomalies_lof / len(df_enriched) * 100).round(2)

print(f"\n✅ Détection terminée")
print(f"🔴 Anomalies : {n_anomalies_lof:,} ({pct_lof}%)")
print(f"🟢 Normaux   : {len(df_enriched) - n_anomalies_lof:,} ({100-pct_lof}%)")

## 6.3 One-Class SVM

In [ ]:
print("="*80)
print(" "*25 + "🎯 ONE-CLASS SVM")
print("="*80)

# Modèle One-Class SVM
oc_svm = OneClassSVM(
    nu=CONTAMINATION,  # nu ~ contamination
    kernel='rbf',
    gamma='auto'
)

# Prédiction
predictions_svm = oc_svm.fit_predict(df_scaled)
scores_svm = -oc_svm.score_samples(df_scaled)

# Résultats
df_enriched['svm_anomaly'] = (predictions_svm == -1).astype(int)
df_enriched['svm_score'] = scores_svm

n_anomalies_svm = df_enriched['svm_anomaly'].sum()
pct_svm = (n_anomalies_svm / len(df_enriched) * 100).round(2)

print(f"\n✅ Détection terminée")
print(f"🔴 Anomalies : {n_anomalies_svm:,} ({pct_svm}%)")
print(f"🟢 Normaux   : {len(df_enriched) - n_anomalies_svm:,} ({100-pct_svm}%)")

## 6.4 Consensus Multi-Algorithmes

In [ ]:
print("="*80)
print(" "*22 + "🎯 CONSENSUS MULTI-ALGORITHMES")
print("="*80)

# Comptage du nombre d'algorithmes détectant une anomalie
df_enriched['n_algos_detecting'] = (
    df_enriched['if_anomaly'] +
    df_enriched['lof_anomaly'] +
    df_enriched['svm_anomaly']
)

# Catégorisation
df_enriched['anomaly_level'] = df_enriched['n_algos_detecting'].map({
    0: 'Normal',
    1: 'Anomalie_Faible',
    2: 'Anomalie_Modérée',
    3: 'Anomalie_Forte'
})

# Score composite (moyenne des scores normalisés)
if_norm = (df_enriched['if_score'] - df_enriched['if_score'].min()) / (df_enriched['if_score'].max() - df_enriched['if_score'].min())
lof_norm = (df_enriched['lof_score'] - df_enriched['lof_score'].min()) / (df_enriched['lof_score'].max() - df_enriched['lof_score'].min())
svm_norm = (df_enriched['svm_score'] - df_enriched['svm_score'].min()) / (df_enriched['svm_score'].max() - df_enriched['svm_score'].min())

df_enriched['composite_score'] = (if_norm + lof_norm + svm_norm) / 3

# Statistiques
print("\n📊 Répartition par niveau :")
level_counts = df_enriched['anomaly_level'].value_counts()
level_pcts = (level_counts / len(df_enriched) * 100).round(2)

summary = pd.DataFrame({
    'Nombre': level_counts,
    'Pourcentage': level_pcts
})
display(summary)

# Matrice de concordance
print("\n📋 Concordance entre algorithmes :")
print(f"IF vs LOF : {((df_enriched['if_anomaly'] == df_enriched['lof_anomaly']).sum() / len(df_enriched) * 100):.1f}%")
print(f"IF vs SVM : {((df_enriched['if_anomaly'] == df_enriched['svm_anomaly']).sum() / len(df_enriched) * 100):.1f}%")
print(f"LOF vs SVM: {((df_enriched['lof_anomaly'] == df_enriched['svm_anomaly']).sum() / len(df_enriched) * 100):.1f}%")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution par niveau
level_counts.plot(kind='bar', ax=axes[0], color=['lightgreen', 'yellow', 'orange', 'red'])
axes[0].set_title('Distribution par niveau d\'anomalie')
axes[0].set_ylabel('Nombre de DAB')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)

# Distribution du score composite
for level in ['Normal', 'Anomalie_Faible', 'Anomalie_Modérée', 'Anomalie_Forte']:
    if level in df_enriched['anomaly_level'].values:
        subset = df_enriched[df_enriched['anomaly_level'] == level]['composite_score']
        axes[1].hist(subset, bins=30, alpha=0.6, label=level)
axes[1].set_xlabel('Score composite')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution du score composite')
axes[1].legend()

plt.tight_layout()
plt.show()

## 6.5 Visualisation PCA

In [ ]:
# Réduction de dimension avec PCA
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(df_scaled)

df_enriched['PC1'] = X_pca[:, 0]
df_enriched['PC2'] = X_pca[:, 1]

var_exp = pca.explained_variance_ratio_
print(f"\n📈 Variance expliquée : PC1={var_exp[0]*100:.1f}% | PC2={var_exp[1]*100:.1f}% | Total={sum(var_exp)*100:.1f}%")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Par niveau d'anomalie
colors_map = {'Normal': 'green', 'Anomalie_Faible': 'yellow',
              'Anomalie_Modérée': 'orange', 'Anomalie_Forte': 'red'}
for level, color in colors_map.items():
    mask = df_enriched['anomaly_level'] == level
    if mask.any():
        axes[0].scatter(df_enriched.loc[mask, 'PC1'],
                       df_enriched.loc[mask, 'PC2'],
                       c=color, alpha=0.5, s=30, label=level)
axes[0].set_xlabel(f'PC1 ({var_exp[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({var_exp[1]*100:.1f}%)')
axes[0].set_title('Anomalies dans l\'espace PCA')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Score composite
scatter = axes[1].scatter(df_enriched['PC1'], df_enriched['PC2'],
                         c=df_enriched['composite_score'],
                         cmap='RdYlGn_r', alpha=0.6, s=30)
axes[1].set_xlabel(f'PC1 ({var_exp[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({var_exp[1]*100:.1f}%)')
axes[1].set_title('Score composite')
plt.colorbar(scatter, ax=axes[1], label='Score')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
# 💡 SECTION 7 : Interprétabilité et Explications

In [ ]:
print("="*80)
print(" "*20 + "💡 GÉNÉRATION DES EXPLICATIONS")
print("="*80)

# Calcul des percentiles de référence
percentiles = {}
ref_cols = ['montant_moyen_par_retrait', 'montant_total', 'nb_total_de_retraits', 'ratio_observe_vs_attendu']
ref_cols = [c for c in ref_cols if c in df_enriched.columns]

for col in ref_cols:
    percentiles[(col, 'p05')] = df_enriched[col].quantile(0.05)
    percentiles[(col, 'p95')] = df_enriched[col].quantile(0.95)

# Fonction d'explication
def generate_explanation(row):
    reasons = []

    # Montant moyen par retrait
    if 'montant_moyen_par_retrait' in row.index and pd.notna(row['montant_moyen_par_retrait']):
        if row['montant_moyen_par_retrait'] < percentiles.get(('montant_moyen_par_retrait','p05'), 0):
            reasons.append("Montant moyen très faible (fractionnement suspect)")
        elif row['montant_moyen_par_retrait'] > percentiles.get(('montant_moyen_par_retrait','p95'), float('inf')):
            reasons.append("Montant moyen très élevé (retraits unitaires atypiques)")

    # Volume total
    if 'montant_total' in row.index and pd.notna(row['montant_total']):
        if row['montant_total'] < percentiles.get(('montant_total','p05'), 0):
            reasons.append("Volume total très faible")
        elif row['montant_total'] > percentiles.get(('montant_total','p95'), float('inf')):
            reasons.append("Volume total très élevé")

    # Nombre de retraits
    if 'nb_total_de_retraits' in row.index and pd.notna(row['nb_total_de_retraits']):
        if row['nb_total_de_retraits'] < percentiles.get(('nb_total_de_retraits','p05'), 0):
            reasons.append("Très faible activité transactionnelle")
        elif row['nb_total_de_retraits'] > percentiles.get(('nb_total_de_retraits','p95'), float('inf')):
            reasons.append("Activité transactionnelle très élevée")

    # Type de site
    if 'dab_hos_site' in row.index and str(row['dab_hos_site']).upper() == 'H':
        reasons.append("DAB hors site (profil de risque différent)")

    # Activité
    if 'is_low_activity' in row.index and row['is_low_activity'] == 1:
        reasons.append("DAB à très faible activité (P10)")
    if 'is_high_activity' in row.index and row['is_high_activity'] == 1:
        reasons.append("DAB à très forte activité (P90)")

    return " | ".join(reasons) if reasons else "Profil rare multi-dimensionnel"

# Générer explications pour les anomalies
df_enriched['explanation'] = df_enriched.apply(
    lambda row: generate_explanation(row) if row['anomaly_level'] != 'Normal' else '',
    axis=1
)

print("\n✅ Explications générées pour toutes les anomalies")

In [ ]:
# Top anomalies avec explications
print("="*80)
print(" "*22 + "🔝 TOP 50 ANOMALIES DÉTECTÉES")
print("="*80)

anomalies = df_enriched[df_enriched['anomaly_level'] != 'Normal'].copy()
print(f"\n📊 Total anomalies : {len(anomalies):,}")
print(f"   - Fortes   : {(df_enriched['anomaly_level'] == 'Anomalie_Forte').sum():,}")
print(f"   - Modérées : {(df_enriched['anomaly_level'] == 'Anomalie_Modérée').sum():,}")
print(f"   - Faibles  : {(df_enriched['anomaly_level'] == 'Anomalie_Faible').sum():,}")

if len(anomalies) > 0:
    # Trier par score composite
    anomalies_sorted = anomalies.sort_values('composite_score', ascending=False)

    # Colonnes à afficher
    display_cols = [
        'num_automate', 'lib_site_implementation', 'code_banque', 'type_carte',
        'montant_total', 'nb_total_de_retraits', 'montant_moyen_par_retrait',
        'dab_hos_site', 'anomaly_level', 'composite_score', 'n_algos_detecting', 'explanation'
    ]
    display_cols = [c for c in display_cols if c in anomalies_sorted.columns]

    print("\n📋 TOP 50 ANOMALIES (triées par score décroissant) :")
    pd.set_option('display.max_colwidth', 100)
    display(anomalies_sorted[display_cols].head(50))
    pd.set_option('display.max_colwidth', 50)

    # Statistiques sur les raisons
    print("\n📊 Raisons d'anomalies les plus fréquentes :")
    all_reasons = []
    for expl in anomalies['explanation']:
        if pd.notna(expl) and expl:
            all_reasons.extend([r.strip() for r in expl.split('|')])

    if all_reasons:
        reason_counts = pd.Series(all_reasons).value_counts().head(10)
        reason_pcts = (reason_counts / len(anomalies) * 100).round(2)

        reason_summary = pd.DataFrame({
            'Nombre': reason_counts,
            'Pourcentage': reason_pcts
        })
        display(reason_summary)

---
# ✅ SECTION 8 : Validation Statistique

In [ ]:
print("="*80)
print(" "*20 + "📊 VALIDATION STATISTIQUE")
print("="*80)

# Comparaison distributions
val_cols = [c for c in ['montant_total', 'nb_total_de_retraits',
                        'montant_moyen_par_retrait', 'ratio_observe_vs_attendu']
           if c in df_enriched.columns]

if val_cols and len(anomalies) > 0:
    normal_data = df_enriched[df_enriched['anomaly_level'] == 'Normal']

    comparison = pd.DataFrame({
        'mean_normal': normal_data[val_cols].mean(),
        'mean_anomaly': anomalies[val_cols].mean(),
        'median_normal': normal_data[val_cols].median(),
        'median_anomaly': anomalies[val_cols].median(),
        'p95_normal': normal_data[val_cols].quantile(0.95),
        'p95_anomaly': anomalies[val_cols].quantile(0.95)
    })

    print("\n📈 Comparaison Normal vs Anomalie :")
    display(comparison)

    # Tests statistiques (Mann-Whitney U)
    print("\n🔬 Tests statistiques (Mann-Whitney U) :")
    print("H0: Distributions identiques | H1: Distributions différentes\n")

    test_results = []
    for col in val_cols:
        try:
            statistic, p_value = stats.mannwhitneyu(
                normal_data[col].dropna(),
                anomalies[col].dropna(),
                alternative='two-sided'
            )
            test_results.append({
                'Variable': col,
                'p-value': p_value,
                'Significatif (α=0.05)': 'Oui ✓' if p_value < 0.05 else 'Non ✗'
            })
        except:
            pass

    if test_results:
        test_df = pd.DataFrame(test_results)
        display(test_df)

        sig_count = (test_df['p-value'] < 0.05).sum()
        print(f"\n✅ {sig_count}/{len(test_results)} variables avec différence significative")

In [ ]:
# Test de stabilité
print("\n" + "="*80)
print(" "*22 + "🔄 TEST DE STABILITÉ")
print("="*80)

def get_top_anomalies_by_contamination(contamination, top_n=30):
    iso_tmp = IsolationForest(
        n_estimators=500,
        contamination=contamination,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    pred_tmp = iso_tmp.fit_predict(df_scaled)
    score_tmp = -iso_tmp.score_samples(df_scaled)

    tmp_df = df_enriched.copy()
    tmp_df['score_tmp'] = score_tmp

    if 'num_automate' in tmp_df.columns:
        return set(tmp_df.nlargest(top_n, 'score_tmp')['num_automate'].values)
    else:
        return set(tmp_df.nlargest(top_n, 'score_tmp').index)

# Test avec différents niveaux
contamination_levels = [0.03, 0.05, 0.08, 0.10]
anomaly_sets = {}

print(f"\n🎯 Test avec {len(contamination_levels)} niveaux de contamination\n")
for cont in contamination_levels:
    anomaly_sets[cont] = get_top_anomalies_by_contamination(cont, top_n=30)
    print(f"   {cont*100}% : {len(anomaly_sets[cont])} anomalies")

# Anomalies core (détectées par tous)
core_anomalies = set.intersection(*anomaly_sets.values())
print(f"\n🎯 Anomalies CORE (détectées à tous les niveaux) : {len(core_anomalies)}")

# Intersection
print("\n🔗 Intersections :")
print(f"   3% ∩ 5%  : {len(anomaly_sets[0.03] & anomaly_sets[0.05])}")
print(f"   5% ∩ 8%  : {len(anomaly_sets[0.05] & anomaly_sets[0.08])}")
print(f"   8% ∩ 10% : {len(anomaly_sets[0.08] & anomaly_sets[0.10])}")

print("\n💡 Les anomalies CORE sont les plus robustes et critiques")

---
# 💾 SECTION 9 : Export des Résultats

⚠️ **ACTION REQUISE** : Créez d'abord un dataset vide dans Dataiku, puis modifiez `OUTPUT_DATASET_NAME`

In [ ]:
# Préparation du dataset d'export
export_cols = [
    # Identifiants
    'num_automate', 'lib_site_implementation', 'code_banque', 'code_postale_emplacement',
    # Variables métier
    'type_carte', 'montant_total', 'nb_total_de_retraits', 'dab_hos_site',
    # Features engineered
    'montant_moyen_par_retrait', 'ratio_montant_vs_median', 'ratio_observe_vs_attendu',
    # Résultats détection
    'if_anomaly', 'lof_anomaly', 'svm_anomaly', 'anomaly_level', 'composite_score',
    'if_score', 'lof_score', 'svm_score', 'n_algos_detecting',
    # Explications
    'explanation'
]

export_cols = [c for c in export_cols if c in df_enriched.columns]
df_export = df_enriched[export_cols].copy()

# Renommer pour clarté
rename_map = {
    'if_anomaly': 'anomaly_isolation_forest',
    'lof_anomaly': 'anomaly_lof',
    'svm_anomaly': 'anomaly_svm',
    'anomaly_level': 'niveau_anomalie',
    'composite_score': 'score_composite',
    'explanation': 'explication'
}
df_export.rename(columns=rename_map, inplace=True)

print("📦 Dataset d'export préparé")
print(f"📏 Dimensions : {df_export.shape[0]:,} × {df_export.shape[1]}")
print(f"\n📋 Colonnes exportées ({len(df_export.columns)}) :")
for i, col in enumerate(df_export.columns, 1):
    print(f"  {i:2d}. {col}")

In [ ]:
# Export vers Dataiku
OUTPUT_DATASET_NAME = "DAB_ANOMALIES_RESULTATS"  # ⚠️ MODIFIER ICI

try:
    output_dataset = dataiku.Dataset(OUTPUT_DATASET_NAME)
    output_dataset.write_with_schema(df_export)

    print(f"\n✅ Export réussi vers '{OUTPUT_DATASET_NAME}'")
    print(f"📊 {len(df_export):,} lignes exportées")
    print(f"\n📈 Statistiques :")
    print(f"   - Anomalies fortes   : {(df_export['niveau_anomalie'] == 'Anomalie_Forte').sum():,}")
    print(f"   - Anomalies modérées : {(df_export['niveau_anomalie'] == 'Anomalie_Modérée').sum():,}")
    print(f"   - Anomalies faibles  : {(df_export['niveau_anomalie'] == 'Anomalie_Faible').sum():,}")
    print(f"   - Normaux            : {(df_export['niveau_anomalie'] == 'Normal').sum():,}")

except Exception as e:
    print(f"\n❌ Erreur d'export : {e}")
    print("\n💡 Solutions :")
    print("   1. Créez le dataset dans Dataiku")
    print("   2. Vérifiez le nom du dataset")
    print("   3. Vérifiez vos permissions")
    print("\n📦 Les résultats restent disponibles dans df_export")

---
# 📊 SECTION 10 : Résumé Exécutif et Recommandations

In [ ]:
print("="*80)
print(" "*15 + "📊 RÉSUMÉ EXÉCUTIF - DÉTECTION D'ANOMALIES DAB")
print("="*80)

# 1. Volumétrie
print("\n1️⃣ VOLUMÉTRIE")
print("="*60)
print(f"Total DAB analysés           : {len(df):,}")
print(f"Features utilisées           : {len(model_features)}")
print(f"Algorithmes de détection     : 3 (IF + LOF + One-Class SVM)")

# 2. Résultats
print("\n2️⃣ RÉSULTATS DE DÉTECTION")
print("="*60)
n_forte = (df_enriched['anomaly_level'] == 'Anomalie_Forte').sum()
n_moderee = (df_enriched['anomaly_level'] == 'Anomalie_Modérée').sum()
n_faible = (df_enriched['anomaly_level'] == 'Anomalie_Faible').sum()
n_normal = (df_enriched['anomaly_level'] == 'Normal').sum()

print(f"🔴 Anomalies FORTES (3 algos)   : {n_forte:,} ({n_forte/len(df)*100:.2f}%)")
print(f"🟠 Anomalies MODÉRÉES (2 algos) : {n_moderee:,} ({n_moderee/len(df)*100:.2f}%)")
print(f"🟡 Anomalies FAIBLES (1 algo)   : {n_faible:,} ({n_faible/len(df)*100:.2f}%)")
print(f"🟢 DAB NORMAUX                  : {n_normal:,} ({n_normal/len(df)*100:.2f}%)")

# 3. Principaux patterns
print("\n3️⃣ PATTERNS D'ANOMALIES IDENTIFIÉS")
print("="*60)
if 'all_reasons' in locals() and all_reasons:
    top_reasons = pd.Series(all_reasons).value_counts().head(5)
    for i, (reason, count) in enumerate(top_reasons.items(), 1):
        pct = (count / len(anomalies) * 100).round(1)
        print(f"{i}. {reason}")
        print(f"   → {count:,} DAB ({pct}% des anomalies)\n")

# 4. Stabilité
print("4️⃣ ROBUSTESSE DES DÉTECTIONS")
print("="*60)
if 'core_anomalies' in locals():
    print(f"Anomalies CORE (stables) : {len(core_anomalies)} DAB")
    print(f"→ Détectées peu importe le paramétrage (haute confiance)")

# 5. Recommandations
print("\n" + "="*80)
print("5️⃣ RECOMMANDATIONS OPÉRATIONNELLES")
print("="*80)
print("""
🎯 PRIORISATION DES ACTIONS :

PRIORITÉ 1 - Anomalies FORTES (consensus 3 algorithmes)
   ├─ Investigation immédiate obligatoire
   ├─ Vérification logs système et historique
   ├─ Audit de sécurité si suspicion fraude
   └─ Blocage préventif si risque élevé

PRIORITÉ 2 - Anomalies CORE (stables)
   ├─ Anomalies persistantes = action requise
   ├─ Analyse des causes racines
   └─ Plan d'action correctif

PRIORITÉ 3 - Anomalies MODÉRÉES (2 algorithmes)
   ├─ Surveillance renforcée
   ├─ Investigation si récurrence
   └─ Catégorisation métier

PRIORITÉ 4 - Anomalies FAIBLES (1 algorithme)
   ├─ Monitoring passif
   └─ Escalade si évolution vers niveau supérieur

🔄 ACTIONS DE SUIVI :

1. Monitoring continu
   └─ Ré-exécution hebdomadaire/mensuelle de l'analyse

2. Enrichissement des données
   ├─ Ajouter données temporelles (jour/heure/saison)
   ├─ Intégrer géolocalisation enrichie
   ├─ Historiser pour détection de tendances
   └─ Ajouter données externes (météo, événements)

3. Feedback métier
   ├─ Valider anomalies avec équipes terrain
   ├─ Documenter faux positifs
   ├─ Ajuster seuils si nécessaire
   └─ Créer taxonomie des anomalies

4. Amélioration continue
   ├─ Tester algorithmes supplémentaires (DBSCAN, Autoencoder)
   ├─ Feature engineering avancé (réseaux, saisonnalité)
   ├─ Système d'alerting automatique
   └─ Dashboard opérationnel temps réel

💡 INDICATEURS CLÉS À SURVEILLER :

- Taux de confirmation des anomalies par les équipes métier
- Temps moyen de résolution par niveau d'anomalie
- Impact financier des anomalies détectées
- Évolution du nombre d'anomalies dans le temps
""")

print("="*80)
print(" "*25 + "✅ ANALYSE TERMINÉE")
print("="*80)
print(f"\n📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Résultats : df_export ({len(df_export):,} lignes)")
print(f"📁 Dataset Dataiku : {OUTPUT_DATASET_NAME}")
print("\n🎉 Analyse complète et validée !")
print("\n💡 Prochaines étapes suggérées :")
print("   1. Examiner les top anomalies avec les équipes métier")
print("   2. Valider les explications générées")
print("   3. Créer un plan d'action par niveau d'anomalie")
print("   4. Programmer l'exécution automatique hebdomadaire")
print("   5. Créer un dashboard de suivi des anomalies")

---

# 📝 Notes Techniques

## Méthodologie

Cette analyse implémente une approche **multi-algorithmes robuste** :

### ✅ Points Forts

1. **Validation croisée** : 3 algorithmes complémentaires (IF + LOF + One-Class SVM)
2. **Feature engineering** : 10+ features métier créées
3. **Normalisation robuste** : RobustScaler résistant aux outliers
4. **Tests de stabilité** : Vérification robustesse des détections
5. **Validation statistique** : Tests Mann-Whitney U
6. **Interprétabilité** : Explications automatiques par anomalie

### 🔧 Paramètres Ajustables

- `CONTAMINATION` : Taux d'anomalies attendu (défaut 5%)
- `N_NEIGHBORS` (LOF) : Nombre de voisins (défaut 20)
- `nu` (One-Class SVM) : Borne supérieure fraction outliers
- Features sélectionnées : Adapter selon données disponibles

### 📚 Pour Aller Plus Loin

**Améliorations possibles** :

1. **Analyse temporelle** : Intégrer séries temporelles et saisonnalité
2. **Clustering** : Segmenter DAB avant détection (urbain/rural)
3. **Deep Learning** : Autoencoders pour patterns complexes
4. **Graphes** : Analyser réseaux de DAB géographiques
5. **Ensemble stacking** : Combiner prédictions de manière optimale

---

**Auteur** : Data Science Team  
**Version** : 2.0  
**Contact** : Pour questions ou améliorations